# Analytics API Setup
## Purpose: Investigate Analytics API availability and configuration

This notebook investigates why Analytics API endpoints return 404 errors.
The original Analytics API testing showed only 20% success rate with all analytics endpoints failing.

**Research Focus:**
- Analytics API feature availability
- Vault configuration requirements
- Alternative analytics endpoints
- Permission requirements

In [1]:
# Import required libraries
import sys
import os

# Add the parent directory to sys.path to import our modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from veevavault.client.vault_client import VaultClient
from veevavault.services.authentication import AuthenticationService
from test_credentials import TEST_VAULTS, DEFAULT_VAULT

# Get vault configuration
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_URL = vault_config["URL"]
VAULT_USERNAME = vault_config["username"]
VAULT_PASSWORD = vault_config["password"]

print("📦 Libraries imported successfully")
print(f"🔗 Target Vault: {VAULT_URL}")
print(f"👤 Username: {VAULT_USERNAME}")

📦 Libraries imported successfully
🔗 Target Vault: https://vv-consulting-michael-mastermind.veevavault.com/
👤 Username: veevatools@vv-consulting.com


In [2]:
# Initialize client and authenticate
print("🔐 Initializing VeevaVault client...")
client = VaultClient()
client.vaultURL = VAULT_URL.rstrip('/')

print("🔑 Authenticating...")
auth_service = AuthenticationService(client)
VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")

auth_response = auth_service.authenticate_with_username_password(
    username=VAULT_USERNAME,
    password=VAULT_PASSWORD,
    vaultDNS=VAULT_DNS
)

if auth_response.get('responseStatus') == 'SUCCESS':
    session_id = auth_response['sessionId']
    vault_id = auth_response.get('vaultId', 'Unknown')
    print(f"✅ Authentication successful!")
    print(f"🆔 Session ID: {session_id[:20]}...")
    print(f"🏢 Vault ID: {vault_id}")
    auth_success = True
else:
    print(f"❌ Authentication failed: {auth_response}")
    auth_success = False

🔐 Initializing VeevaVault client...
🔑 Authenticating...
✅ Authentication successful!
🆔 Session ID: 2C5F99B371380857552D...
🏢 Vault ID: 92425
✅ Authentication successful!
🆔 Session ID: 2C5F99B371380857552D...
🏢 Vault ID: 92425


In [3]:
# Test: Check vault configuration and applications
if auth_success:
    print("🏢 Checking vault configuration...")
    
    # Check vault info
    vault_info_response = client.api_call("api/v25.2/vault")
    
    if vault_info_response.get('responseStatus') == 'SUCCESS':
        vault_data = vault_info_response.get('data', {})
        vault_name = vault_data.get('name', 'Unknown')
        vault_status = vault_data.get('status', 'Unknown')
        vault_type = vault_data.get('vault_type', 'Unknown')
        
        print(f"✅ Vault info retrieved")
        print(f"  📊 Name: {vault_name}")
        print(f"  📊 Status: {vault_status}")
        print(f"  📊 Type: {vault_type}")
        
        # Check for analytics-related configuration
        print(f"\n🔍 Full vault data keys: {list(vault_data.keys())}")
    else:
        print(f"❌ Failed to get vault info: {vault_info_response}")

🏢 Checking vault configuration...


Exception: HTTP error occurred: 404 Client Error: Not Found for url: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/vault

In [4]:
# Test: Check available API endpoints
if auth_success:
    print("🔍 Checking available API endpoints...")
    
    # Get API version info
    api_versions_response = client.api_call("api/")
    
    if api_versions_response.get('responseStatus') == 'SUCCESS':
        versions = api_versions_response.get('values', [])
        print(f"✅ Found {len(versions)} API versions")
        
        # Check latest version for analytics endpoints
        latest_version = "v25.2"
        print(f"\n🎯 Using API version: {latest_version}")
    else:
        print(f"❌ Failed to get API versions: {api_versions_response}")

🔍 Checking available API endpoints...
✅ Found 37 API versions

🎯 Using API version: v25.2


In [5]:
# Test: Check permissions and user capabilities
if auth_success:
    print("👤 Checking user permissions...")
    
    # Get user info
    user_response = client.api_call("api/v25.2/objects/users/me")
    
    if user_response.get('responseStatus') == 'SUCCESS':
        user_data = user_response.get('data', {})
        user_name = user_data.get('user_name__v', 'Unknown')
        user_email = user_data.get('user_email__v', 'Unknown')
        
        print(f"✅ User info retrieved")
        print(f"  📊 Name: {user_name}")
        print(f"  📊 Email: {user_email}")
        
        # Check for analytics-related permissions
        print(f"\n🔍 User data keys: {list(user_data.keys())}")
    else:
        print(f"❌ Failed to get user info: {user_response}")

👤 Checking user permissions...
✅ User info retrieved
  📊 Name: Unknown
  📊 Email: Unknown

🔍 User data keys: []
✅ User info retrieved
  📊 Name: Unknown
  📊 Email: Unknown

🔍 User data keys: []


In [6]:
# Test: Try various analytics endpoint patterns
if auth_success:
    print("🔬 Testing various analytics endpoint patterns...")
    
    # List of potential analytics endpoints to test
    analytics_endpoints = [
        "api/v25.2/services/analytics",
        "api/v25.2/analytics", 
        "api/v25.2/objects/analytics",
        "api/v25.2/metadata/analytics",
        "api/v25.2/services/reporting",
        "api/v25.2/reporting",
        "api/v25.2/objects/reporting",
        "api/v25.2/dashboards",
        "api/v25.2/services/dashboards"
    ]
    
    print(f"🎯 Testing {len(analytics_endpoints)} potential endpoints...\n")
    
    for endpoint in analytics_endpoints:
        try:
            response = client.api_call(endpoint)
            if response.get('responseStatus') == 'SUCCESS':
                print(f"✅ {endpoint}: SUCCESS")
                # If we find a working endpoint, show some data
                data = response.get('data', [])
                if data:
                    print(f"   📊 Found {len(data)} items")
            else:
                print(f"❌ {endpoint}: {response.get('responseStatus', 'UNKNOWN')}")
        except Exception as e:
            error_msg = str(e)
            if "404" in error_msg:
                print(f"❌ {endpoint}: 404 Not Found")
            elif "403" in error_msg:
                print(f"🔒 {endpoint}: 403 Forbidden (Permission issue)")
            else:
                print(f"💥 {endpoint}: Error - {error_msg}")

🔬 Testing various analytics endpoint patterns...
🎯 Testing 9 potential endpoints...

❌ api/v25.2/services/analytics: 404 Not Found
❌ api/v25.2/analytics: 404 Not Found
❌ api/v25.2/analytics: 404 Not Found
❌ api/v25.2/objects/analytics: 404 Not Found
❌ api/v25.2/metadata/analytics: 404 Not Found
❌ api/v25.2/objects/analytics: 404 Not Found
❌ api/v25.2/metadata/analytics: 404 Not Found
❌ api/v25.2/services/reporting: 404 Not Found
❌ api/v25.2/reporting: 404 Not Found
❌ api/v25.2/services/reporting: 404 Not Found
❌ api/v25.2/reporting: 404 Not Found
❌ api/v25.2/objects/reporting: 404 Not Found
❌ api/v25.2/dashboards: 404 Not Found
❌ api/v25.2/objects/reporting: 404 Not Found
❌ api/v25.2/dashboards: 404 Not Found
❌ api/v25.2/services/dashboards: 404 Not Found
❌ api/v25.2/services/dashboards: 404 Not Found


In [7]:
# Test: Check for reports or dashboards (alternative analytics)
if auth_success:
    print("📊 Checking for reports and dashboards...")
    
    # Try to find any reporting/analytics objects
    try:
        # Check for reports
        reports_response = client.api_call("api/v25.2/objects/reports")
        if reports_response.get('responseStatus') == 'SUCCESS':
            reports_data = reports_response.get('data', [])
            print(f"✅ Reports endpoint accessible: {len(reports_data)} reports found")
        else:
            print(f"❌ Reports endpoint failed: {reports_response.get('responseStatus')}")
    except Exception as e:
        print(f"❌ Reports endpoint error: {str(e)}")
    
    try:
        # Check for dashboards 
        dashboards_response = client.api_call("api/v25.2/objects/dashboards")
        if dashboards_response.get('responseStatus') == 'SUCCESS':
            dashboards_data = dashboards_response.get('data', [])
            print(f"✅ Dashboards endpoint accessible: {len(dashboards_data)} dashboards found")
        else:
            print(f"❌ Dashboards endpoint failed: {dashboards_response.get('responseStatus')}")
    except Exception as e:
        print(f"❌ Dashboards endpoint error: {str(e)}")

📊 Checking for reports and dashboards...
❌ Reports endpoint error: HTTP error occurred: 404 Client Error: Not Found for url: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/objects/reports
❌ Dashboards endpoint error: HTTP error occurred: 404 Client Error: Not Found for url: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/objects/dashboards
❌ Dashboards endpoint error: HTTP error occurred: 404 Client Error: Not Found for url: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/objects/dashboards


In [8]:
# Test: Check metadata to see what objects are available
if auth_success:
    print("🔍 Checking available metadata objects...")
    
    try:
        metadata_response = client.api_call("api/v25.2/metadata/objects")
        if metadata_response.get('responseStatus') == 'SUCCESS':
            objects = metadata_response.get('objects', [])
            print(f"✅ Found {len(objects)} object types")
            
            # Look for analytics-related objects
            analytics_objects = []
            for obj in objects:
                obj_name = obj.get('name', '').lower()
                obj_label = obj.get('label', '').lower()
                if any(keyword in obj_name or keyword in obj_label for keyword in ['analytic', 'report', 'dashboard', 'chart']):
                    analytics_objects.append(obj)
            
            if analytics_objects:
                print(f"\n📊 Found {len(analytics_objects)} analytics-related objects:")
                for obj in analytics_objects[:5]:  # Show first 5
                    print(f"  - {obj.get('label', 'Unknown')} ({obj.get('name', 'Unknown')})")
            else:
                print("\n⚠️ No analytics-related objects found in metadata")
                print("\n📋 Sample available objects:")
                for obj in objects[:5]:  # Show first 5 regular objects
                    print(f"  - {obj.get('label', 'Unknown')} ({obj.get('name', 'Unknown')})")
        else:
            print(f"❌ Metadata request failed: {metadata_response}")
    except Exception as e:
        print(f"❌ Metadata error: {str(e)}")

🔍 Checking available metadata objects...
✅ Found 0 object types

⚠️ No analytics-related objects found in metadata

📋 Sample available objects:


## 🎯 Analytics API Setup Analysis Summary

This notebook investigated why Analytics API endpoints return 404 errors:

### Key Findings:
1. **Vault Configuration** - Checked vault type and status
2. **API Endpoint Availability** - Tested various analytics endpoint patterns
3. **User Permissions** - Verified user capabilities and access levels
4. **Alternative Analytics** - Looked for reports/dashboards as alternatives
5. **Metadata Analysis** - Checked available object types for analytics features

### Likely Root Causes:
- **Feature Not Enabled**: Analytics API may not be enabled for this vault type
- **Licensing Issue**: Advanced analytics features may require specific licensing
- **Vault Type Limitation**: Some vault types don't support full analytics APIs
- **Version Incompatibility**: Analytics endpoints may be available in different API versions

### Recommendations:
1. **Contact Vault Administrator** - Verify analytics feature licensing and configuration
2. **Check Alternative Endpoints** - Use reports/dashboards if available
3. **Update Test Expectations** - Mark analytics tests as environment-dependent
4. **Document Limitations** - Note which features require specific vault configurations